In [1]:
# IMPORTS
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Other Common Modules
from datetime import datetime, timedelta
import importlib


# IMPORT COMPLETE
print("Imports Done")

Imports Done


In [2]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    timenow = datetime.now()+timedelta(hours=8)
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir("/content/drive/MyDrive/Chronical/2023Spring/ML_drive/MLHW2")
else:
    print('Running on Local')
    timenow = datetime.now()

BASE_PATH = os.getcwd()
DATA_PATH = os.path.join(BASE_PATH, "data")
SAVE_PATH = os.path.join(BASE_PATH, ".model")
OUTPUT_PATH = os.path.join(BASE_PATH, "output")

if not os.path.isdir(SAVE_PATH):
    os.mkdir(SAVE_PATH)
if not os.path.isdir(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

timenow_str = f"{timenow.hour:02d}{timenow.minute:02d}{timenow.month:02d}{timenow.day:02d}"

print(f"{BASE_PATH=}")
print(f"{DATA_PATH=}")
print(f"{SAVE_PATH=}")
print(f"{OUTPUT_PATH=}")
print(f"{timenow=}")
print(f"{timenow_str}")

Running on Local
BASE_PATH='G:\\My Drive\\Chronical\\2023Spring\\ML_drive\\MLHW2'
DATA_PATH='G:\\My Drive\\Chronical\\2023Spring\\ML_drive\\MLHW2\\data'
SAVE_PATH='G:\\My Drive\\Chronical\\2023Spring\\ML_drive\\MLHW2\\.model'
OUTPUT_PATH='G:\\My Drive\\Chronical\\2023Spring\\ML_drive\\MLHW2\\output'
timenow=datetime.datetime(2023, 7, 9, 21, 57, 51, 745666)
21570709


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    "device": device,
    "seed": 3141592,
    "learning_rate": 1e-5,
    "epochs": 30000,
    "batch_size": 256, # larger?
    "valid_ratio": 0.2,
    "early_stop": 1000,
    "save_path": SAVE_PATH,
    "output_path": OUTPUT_PATH,
    "time_string": timenow_str,
    "load_checkpoint": True,
    "load_name": "11390709"
}
print(f"device: {torch.cuda.get_device_name(device)}")

device: NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [4]:
# RANDOMNESS FIXED
torch.manual_seed(config["seed"])
np.random.seed(config["seed"])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(config["seed"])

print("Randomness Fixed")

Randomness Fixed


# DATASET & DATALOADER

In [5]:
import SoundData
importlib.reload(SoundData)

<module 'SoundData' from 'G:\\My Drive\\Chronical\\2023Spring\\ML_drive\\MLHW2\\SoundData.py'>

In [6]:
sound_data_parser = SoundData.SoundDataParser(DATA_PATH)
train_dict = sound_data_parser.train_dict
test_dict = sound_data_parser.test_dict

print(f"{len(train_dict)=}")
print(f"{len(test_dict)=}")

100%|██████████| 857/857 [00:14<00:00, 57.76it/s, file=test 911-128684-0087.pt] 

len(train_dict)=3429
len(test_dict)=857


In [7]:
train_dataset = SoundData.SoundDataset(train_dict)
test_dataset = SoundData.SoundDataset(test_dict)
print(f"{len(train_dataset)=}")
print(f"{len(test_dataset)=}")

100%|██████████| 857/857 [00:03<00:00, 237.57it/s]

len(train_dataset)=2116794
len(test_dataset)=527364


In [8]:
original_train_length = len(train_dataset)
actual_valid_length = int(original_train_length*config["valid_ratio"])
actual_train_length = original_train_length - actual_valid_length
train_dataset, valid_dataset = random_split(train_dataset, [actual_train_length, actual_valid_length], generator=torch.Generator().manual_seed(config["seed"]))

train_loader = DataLoader(train_dataset, config["batch_size"], shuffle=True)
valid_loader = DataLoader(valid_dataset, config["batch_size"], shuffle=True)
test_loader = DataLoader(test_dataset, config["batch_size"], shuffle=False)

# Nural Network, Criteria, Optimizer

In [9]:
import SoundNetwork
importlib.reload(SoundNetwork)

<module 'SoundNetwork' from 'G:\\My Drive\\Chronical\\2023Spring\\ML_drive\\MLHW2\\SoundNetwork.py'>

In [10]:
model = SoundNetwork.SoundNetwork(5)
if config["load_checkpoint"]:
    model.load_state_dict(torch.load(os.path.join(config["save_path"], f"model_{config['load_name']}.ckpt")))

model = model.to(device)
print(f"{model=}")

model=SoundNetwork(
  (net): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): BasicBlock(
      (block): Sequential(
        (0): Linear(in_features=429, out_features=512, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Dropout(p=0.5, inplace=False)
      )
    )
    (2): BasicBlock(
      (block): Sequential(
        (0): Linear(in_features=512, out_features=256, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Dropout(p=0.5, inplace=False)
      )
    )
    (3): BasicBlock(
      (block): Sequential(
        (0): Linear(in_features=256, out_features=128, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Dropout(p=0.5, inplace=False)
      )
    )
    (4): BasicBlock(
      (block): Sequential(
        (0): L

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
print(f"{criterion=}, {optimizer=}")

criterion=CrossEntropyLoss(), optimizer=Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-05
    maximize: False
    weight_decay: 0
)


# Training

In [12]:
import Trainer
importlib.reload(Trainer)

<module 'Trainer' from 'G:\\My Drive\\Chronical\\2023Spring\\ML_drive\\MLHW2\\Trainer.py'>

In [13]:
trainer = Trainer.Trainer(model, criterion, optimizer, train_loader, valid_loader, config)
trainer.train(config["epochs"])

322/30000:   1%|          | 323/30000 [5:27:34<501:37:30, 60.85s/it, train_loss=3.1777, valid_loss=3.1460, early_countdown=986, accuracy=60.85%] 

KeyboardInterrupt



In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# Inference/Test

In [ ]:
import Inferencer
importlib.reload(Inferencer)

In [ ]:
inferencer = Inferencer.Inferencer(config, SoundNetwork.SoundNetwork, config["load_name"], test_loader)
inferencer.infer()

# Experiments

In [ ]:
import torch
import torch.nn as nn
import numpy as np
x = torch.rand(4,41)
net = nn.Softmax()
x = net(x)
loss_function = nn.CrossEntropyLoss()
y = torch.randint(0, 40, (4,))
loss_function(x,y)